# Pre-processing News Data

## Bài toán
Dữ liệu gồm n văn bản phân vào 10 chủ đề khác nhau. Cần biễu diễn mỗi văn bản dưới dạng một vector số thể hiện cho nội dụng của văn bản đó.

## Mục lục
- Load dữ liệu từ thư mục
- Loại bỏ các stop words
- Sử dụng thư viện để mã hóa TF-IDF cho mỗi văn bản

## Phương pháp mã hóa: TF-IDF
Cho tập gồm $n$ văn bản: $D = \{d_1, d_2, ... d_n\}$. Tập từ điển tương ứng được xây dựng từ $n$ văn bản này có độ dài là $m$
- Xét văn bản $d$ có $|d|$ từ và $t$ là một từ trong $d$. Mã hóa tf-idf của $t$ trong văn bản $d$ được biểu diễn:
\begin{equation}
    \begin{split}
        \text{tf}_{t, d} &= \frac{f_t}{|d|} \\
        \text{idf}_{t, d} &= \log\frac{n}{n_t}, \ \ \ \ n_t = |\{d\in D: t\in d\}| \\
        \text{tf-idf}_{t d} &= \text{tf}_{t, d} \times \text{idf}_{t, d}
    \end{split}
\end{equation}

- Khi đó văn bản $d$ được mã hóa là một vector $m$ chiều. Các từ xuất hiện trong d sẽ được thay bằng giá trị tf-idf tương ứng. Các từ không xuất hiện trong $d$ thì thay là 0

In [40]:
import os
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_files
from pyvi import ViTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

%matplotlib inline

## Load dữ liệu từ thư mục

Cấu trúc thư mục như sau 

- data/news_vnexpress/

    - Kinh tế: 
        - bài báo 1.txt 
        - bài báo 2.txt 
    - Pháp luật
        - bài báo 3.txt 
        - bài báo 4.txt 

In [41]:
INPUT = "news_vnexpress"
os.makedirs("images",exist_ok=True)  # thư mục lưu các các hình ảnh trong quá trình huấn luyện và đánh gía

In [42]:
# statistics
print('Các nhãn và số văn bản tương ứng trong dữ liệu')
print('----------------------------------------------')
n = 0
for label in os.listdir(INPUT):
    label_path = os.path.join(INPUT, label)
    if os.path.isdir(label_path):
        num_files = len(os.listdir(label_path))
        print(f'{label}: {num_files}')
        n += num_files

print('-------------------------')
print(f"Tổng số văn bản: {n}")

Các nhãn và số văn bản tương ứng trong dữ liệu
----------------------------------------------
doi-song: 120
du-lich: 54
giai-tri: 201
giao-duc: 105
khoa-hoc: 144
kinh-doanh: 262
phap-luat: 59
suc-khoe: 162
the-thao: 173
thoi-su: 59
-------------------------
Tổng số văn bản: 1339


In [43]:
# load data
data_train = load_files(container_path=INPUT, encoding="utf-8")
print('mapping:')
for i in range(len(data_train.target_names)):
    print(f'{data_train.target_names[i]} - {i}')

print('--------------------------')
print(data_train.filenames[0:1])
# print(data_train.data[0:1])
print(data_train.target[0:1])
print(data_train.data[0:1])

print("\nTổng số  văn bản: {}" .format( len(data_train.filenames)))

mapping:
doi-song - 0
du-lich - 1
giai-tri - 2
giao-duc - 3
khoa-hoc - 4
kinh-doanh - 5
phap-luat - 6
suc-khoe - 7
the-thao - 8
thoi-su - 9
--------------------------
['news_vnexpress\\khoa-hoc\\00133.txt']
[4]
['Mời độc giả đặt câu hỏi tại đây\n']

Tổng số  văn bản: 1339


## Chuyển dữ liệu dạng text về ma trận (n x m) bằng TF-IDF

* Bạn cần viết đoạn mã tương ứng trong cell bên dưới. Theo các bước được gợi ý

In [64]:
# load dữ liệu các stopwords 

#---> Code ở đây
list_stopwords = ['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó', 'alô', 'amen', 'anh', 'anh_ấy']
count = 0
for i in data_train.data:
    for word in i.split():
        if word in list_stopwords:
            count += 1
print(f'Số lượng stopwords: {count}')
print(list_stopwords)
# Chuyển hoá dữ liệu text về dạng vector TF 
#     - loại bỏ từ dừng
#     - sinh từ điển

#---> Code ở đây
module_count_vector = CountVectorizer(stop_words = list_stopwords)
tf_matrix = module_count_vector.fit_transform(data_train.data)
module_count_vector.get_feature_names_out()




# Hàm thực hiện chuyển đổi dữ liệu text thành dữ liệu số dạng ma trận 
# Input: Dữ liệu 2 chiều dạng numpy.array, mảng nhãn id dạng numpy.array 
tfidf_transformer = TfidfTransformer()
data_preprocessed = tfidf_transformer.fit_transform(tf_matrix)


X = data_preprocessed # thuoc tinh
Y = data_train.target #nhan

print(f"\nSố lượng từ trong từ điển: {len(module_count_vector.vocabulary_)}")
print(f"Kích thước dữ liệu sau khi xử lý: {X.shape}")
print(f"Kích thước nhãn tương ứng: {Y.shape}")

Số lượng stopwords: 1287
['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó', 'alô', 'amen', 'anh', 'anh_ấy']

Số lượng từ trong từ điển: 13108
Kích thước dữ liệu sau khi xử lý: (1339, 13108)
Kích thước nhãn tương ứng: (1339,)


In [61]:
print(X[100].toarray())
print(Y[100])

[[0.         0.         0.         ... 0.         0.11843162 0.        ]]
5


In [62]:
sum(sum(X[100].toarray() != 0))

np.int64(428)

In [63]:
print(X[100])

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 428 stored elements and shape (1, 13108)>
  Coords	Values
  (0, 81)	0.012823375184355683
  (0, 100)	0.017508728791792258
  (0, 156)	0.019610602838140156
  (0, 188)	0.039810664877639734
  (0, 269)	0.028021629709682826
  (0, 392)	0.020892778423825517
  (0, 397)	0.02885838071854897
  (0, 418)	0.04123793055744669
  (0, 662)	0.019195050669718137
  (0, 909)	0.02797263592768561
  (0, 1194)	0.04313657347966995
  (0, 1209)	0.0862731469593399
  (0, 1219)	0.04313657347966995
  (0, 1271)	0.013492809584655142
  (0, 1529)	0.012522770360808119
  (0, 1591)	0.029443144291452707
  (0, 1632)	0.019805400051052673
  (0, 1784)	0.03991191606062433
  (0, 1789)	0.03915646764619453
  (0, 1868)	0.04313657347966995
  (0, 2048)	0.03443464303978419
  (0, 2060)	0.060583285651924244
  (0, 2085)	0.012263714907304922
  (0, 2111)	0.022707659194638526
  (0, 2121)	0.021728860265386903
  :	:
  (0, 12856)	0.02702855594070609
  (0, 12864)	0.028021629709682826
  (0